# L’image dans la machine II

## Du pixel aux images - 32M7138

*Printemps 2025 - Université de Genève*

*Adrien Jeanrenaud (adrien.jeanrenaud@unige.ch)*

## **Plan du cours**

> **Appliquer des transformations à une image**
> * Modification sur un pixel
> * Modification sur plusieurs pixels
> * Notions de bruits et de filtres
> * Exemple d'application : détecter les contours

In [ ]:
# Uniquement pour Google Colab

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import requests 

# Lien de partage
file_id = "1SKRafrz2xDAoBszxotXA0TkPKRqdycct"
download_url = f"https://drive.google.com/uc?export=download&id={file_id}"

# Télécharger l'image --> à vous de jouer
response = ...

In [ ]:
chemin_image = "images/coins.jpg"

### Modification sur un pixel

Une image est composée de pixels, chacun ayant une couleur définie par des valeurs (R, V, B) pour le rouge, le vert et le bleu. Modifier un pixel revient à changer ces valeurs.

**Exemples d'application**

- Changer la couleur d'un pixel (ex: transformer un pixel rouge en bleu).
- Appliquer un effet de luminosité en augmentant les valeurs des pixels.

In [ ]:
# Charger une image
image = cv2.imread(chemin_image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Modifier un pixel (exemple : changer le pixel en haut à gauche en rouge)
image[0,0] = [0, 0, 255]  # Bleu, Vert, Rouge (BGR)

plt.imshow(image[0:10,0:10])
plt.show()

### Modification sur plusieurs pixels

Modifier plusieurs pixels en même temps permet d'effectuer des traitements globaux comme le changement de contraste, la conversion en noir et blanc, etc.

**Exemple d'application**

- Convertir une image en noir et blanc.
- Augmenter ou diminuer la luminosité de toute l'image.

In [ ]:
# Charger une image
image = cv2.imread(chemin_image)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Modifier un pixel (exemple : changer le pixel en haut à gauche en rouge)
image[0:500, 0:500] = [0, 0, 255]  # Bleu, Vert, Rouge (BGR)

plt.imshow(image)
plt.show()

In [ ]:
# Conversion en niveaux de gris
image = cv2.imread(chemin_image)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

plt.imshow(gray, cmap="gray")
plt.show()

#### Exposition et contraste

L'exposition

Il est possible de corriger l'exposition, c'est à dire d'éclaircir ou d'assombrir une image. En utilisant le correction gamma, on contrôle la luminosité en changeant les ratios RGB

In [ ]:
from skimage import exposure

?exposure.adjust_gamma

**En dessous de 1, l'image s'éclaircit, en dessus de 1 elle s'assombrit**

In [ ]:
#image couleur
color_gamma = exposure.adjust_gamma(image, gamma = 2.25)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en couleurs')
ax1.imshow(image)

ax2.set_title ('Image transformée')
ax2.imshow(color_gamma)

**Le contraste**

Le contraste définit la répartition de lumière dans l'image.
Modifier le contraste de l'image permet d'ouvrir la fenêtre des pixels ; si les valeurs min et max ont peu d'écart, il est possible d'augmenter la rangée des valeurs utilisées

Mofifier le contraste à la main

In [ ]:
# Trouver les valeurs min et max 

ma = gray.max()
mi = gray.min()
print(mi,ma)

In [ ]:
# Convertir l'image en float et ouvrir la fenêtre de valeurs

c = gray.astype(float)
gray_c = 255.0*(c-mi)/(ma-mi+0.0000001).astype(int)

In [ ]:
# Est-ce que ça a bien fonctionné ?

ma1 = gray_c.max()
mi1 = gray_c.min()
print(mi1,ma1)

In [ ]:
# Est-ce que ça a bien fonctionné ?

ma1 = gray_c.max()
mi1 = gray_c.min()
print(mi1,ma1)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_c,cmap = "gray")

In [ ]:
# On divise l'image en différents r, g, b 

r,g,b = cv2.split(image)

In [ ]:
ma = r.max()
mi = r.min()
print(mi,ma)
c = r.astype(float)
im1r = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
ma = g.max()
mi = g.min()
print(mi,ma)
c = g.astype(float)
im1g = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
ma = b.max()
mi = b.min()
print(mi,ma)
c = b.astype(float)
im1b = 255.0*(c-mi)/(ma-mi+0.0000001)

In [ ]:
# On remet les canaux ensemble
# Attention à l'ordre des canaux de couleurs

color_c = cv2.merge([im1r, im1g, im1b]).astype(int)


In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image couleur')
ax1.imshow(image)

ax2.set_title ('Image transformée')
ax2.imshow(color_c)

**Seuillage**

Le principe du seuillage (threshold) est de passser d'une image en valeurs de gris à une image (ou une partie de l'image) binaire. Une image binaire signifie que chaque pixel n'a que deux choix pour valeur, le plus souvent noir ou blanc. Le seuillage sert à mettre à jour une partie de l'image, un objet en particulier ou des particularités de l'image. Pour pouvoir appliquer un seuillage à une image, il faut nécessairement choisir un seuil, voire deux, qui diviseront la répartition des pixels

In [ ]:
?cv2.threshold

**Il existe différentes manières de faire du seuillage**

* cv2.THRESH_BINARY
* cv2.THRESH_BINARY_INV
* cv2.THRESH_TRUNC
* cv2.THRESH_TOZERO
* cv2.THRESH_TOZERO_INV

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fwww.scaler.com%2Ftopics%2Fimages%2Fcv2-threshold-1.webp&f=1&nofb=1&ipt=72602e1a26cdda7b6a27d0fc604c13cf5ba92c1225511443d65b3c20b6953337&ipo=images" title="binary"/>

**cv2.THRESH_BINARY**

Cette binarisation est la plus simple. Un seuil détermine les valeurs qui seront mises à 0 ou à 255

In [ ]:
# La fonction prend comme argument: une image, un seuil, une valeur maximale et un type de seuillage
# La fonction retourne: le seuil et l'image binarisée

_, gray_b = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY)

In [ ]:
print("seuil :", _)

In [ ]:
# Afficher les images 

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.title("Image de base")
plt.imshow(gray, cmap="gray")

plt.subplot(212)
plt.title("binary")
plt.imshow(gray_b, cmap="gray")
plt.show()

**cv2.THRESH_BINARY_INV**

Cette binarisation est l'inverse de la précédente. Si la le pixel est supérieur au seuil, alors la valeur malximale lui sera assigné

In [ ]:
# La fonction prend comme argument: une image, un seuil, une valeur maximale et un type de seuillage
# La fonction retourne: le seuil et l'image binarisée

_, gray_bi = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV)

In [ ]:
# Afficher les images 

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.title("Image de base")
plt.imshow(gray, cmap="gray")

plt.subplot(212)
plt.title("binary_inv")
plt.imshow(gray_bi, cmap="gray")
plt.show()

**cv2.THRESH_THRESH_TRUNC**

Ce seuillage permet, par le choix d'un seuil T, de transformer tous les pixels supérieurs à la valeur du seuil tandis que les pixels en dessous du seuil ne sont pas modifiés

In [ ]:
# La fonction prend comme argument: une image, un seuil, une valeur maximale et un type de seuillage
# La fonction retourne: le seuil et l'image binarisée

_, gray_t = cv2.threshold(gray, 100, 255, cv2.THRESH_TRUNC)

In [ ]:
# Afficher les images 

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.title("Image de base")
plt.imshow(gray, cmap="gray")

plt.subplot(212)
plt.title("trunc")
plt.imshow(gray_t, cmap="gray")
plt.show()

**cv2THRESH_TOZERO**

Ici il s'agit de définir un seuil dont les valeurs, si elles sont inférieures au seuil, sont mises à 0 tandis que les valeurs supérieures au seuil ne changent pas.

In [ ]:
# La fonction prend comme argument: une image, un seuil, une valeur maximale et un type de binarisation
# La fonction retourne: le seuil et l'image binarisée

_, gray_z = cv2.threshold(gray, 120, 255, cv2.THRESH_TOZERO)

In [ ]:
# Afficher les images 

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.title("Image de base")
plt.imshow(gray, cmap="gray")

plt.subplot(212)
plt.title("tozero")
plt.imshow(gray_z, cmap="gray")
plt.show()

**cv2.THRESH_TOZERO_INV**

Cette méthode est similaire à la précédente, à la seule différence que les valeurs en dessus du seuil sont cette fois mises à 0

In [ ]:
# La fonction prend comme argument: une image, un seuil, une valeur maximale et un type de binarisation
# La fonction retourne: le seuil et l'image binarisée

_, gray_zi = cv2.threshold(gray, 30, 255, cv2.THRESH_TOZERO_INV)

In [ ]:
# Afficher les images 

plt.figure(figsize=(10, 10))
plt.subplot(211)
plt.title("Image de base")
plt.imshow(gray, cmap="gray")

plt.subplot(212)
plt.title("binary")
plt.imshow(gray_zi, cmap="gray")
plt.show()

In [ ]:
### Thresholding
# 0, binary
# 1, binary inverted
# 2, Truncated
# 3, Threshold to Zero
# 4, Threshold to Zero inverted

_, gray0 = cv2.threshold(gray, 50, 255, 0)
_, gray1 = cv2.threshold(gray, 50, 255, 1)
_, gray2 = cv2.threshold(gray, 50, 255, 2)
_, gray3 = cv2.threshold(gray, 50, 255, 3)
_, gray4 = cv2.threshold(gray, 50, 255, 4)

In [ ]:
titles = ["original", 'BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
images = [gray, gray0, gray1, gray2, gray3, gray4,]
for i in range(6):
    plt.subplot(2,3,i+1)
    plt.imshow(images[i],cmap="gray")
    plt.title(titles[i])
    plt.xticks([]),plt.yticks([])
plt.show()

### Notions de bruits et de filtres

Le bruit est une altération aléatoire qui peut rendre une image floue ou déformée (ex: bruit de caméra, compression d’image). On utilise des filtres pour atténuer ces bruits.

**Exemple d'application**

- Réduction du bruit dans des images médicales.
- Amélioration de la qualité d’une image prise dans de mauvaises conditions.

#### Qu'est-ce qu'un filtre? ?

Dans le traitement numérique des images, les filtres sont utilisés principalement pour flouter, améliorer la netteté ou détecter les contours d'une image. Le filtre permet de supprimer les impurtées, le plus souvent il prépare l'image en vue d'opérations plus poussées, comme pour l'apprentissage profond. 

Un filtre est une petite matrice de dimension impair (3x3, 9x9, etc.) qui s'applique par convolution à l'image. Une convolution est simplement un opération matriciel entre le filtre et l'image. Le filtre (une matrice de 3x3 par exemple), se déplace sur l'image et une nouvelle image est obtenue lorsque l'opération est effectuée sur chaque pixel.
L'opération de convolution réduit la taille de l'image, à moins que des bords soient ajoutés. 

<img src="https://miro.medium.com/v2/0*YfpMfPnz6n2g4vIz.jpg" title="filtre"/>

<img src="https://drek4537l1klr.cloudfront.net/elgendy/Figures/3-13.png" title="filtre2"/>

#### Attention aux bords

Le déplacement du filtre sur l'image pose la question du traitement des bords, car les pixels aux extrémités de l'image doivent avoir le même traitement que les autres ; c'est-à-dire qu'ils doivent passer par toutes les composantes du filtre. Dans ce cas, il y a plusieurs méthodes pour élargir les bords de l'image afin que tous les pixels de l'image de base soient traités correctement. 

Le processus de création de données en dehors de l'image s'appelle en anglais "padding".

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fmiro.medium.com%2Fmax%2F3232%2F1*9reDuDh3nXs_kJ-M4eq0Ow.png&f=1&nofb=1&ipt=5e8431d61861a6afe5ece799945d9a7a3841a1e80ad8eb42c80e8c5b4f567755&ipo=images" title="padding"/>


**Principalement, il y a**
> * ajout de zéros
> * constante arbitraire
> * plus proche voisin
> * en miroir 
> * reprend le bors opposé

#### Du filtre à l'image

Il existe plusieurs types de filtres dont le paramétrage et les effets sont bien connus. Avant de les aborder, regardons comment l'on peut simplement créer et appliquer un fitlre sur notre image.

https://setosa.io/ev/image-kernels/

In [ ]:
# La fonction convolve de la librairie Scipy nous permet d'opérer une convolution sur une image 
# Regardons ses paramètres

import scipy.ndimage

?scipy.ndimage.filters.convolve

**Il nous faut principalement une image (input), un filtre (weights) et un choix de padding (mode)**

In [ ]:
# Créer un filtre, 3x3 par exemple

filtre = np.array([[1,0,1],
                  [0,1,0],
                  [1,0,1]])
print(filtre.shape, "\n", filtre)

In [ ]:
# Paramétrer la convolution

grayf = gray.copy()

gray_filtre = scipy.ndimage.filters.convolve(grayf, filtre, mode="reflect")

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_filtre,cmap = "gray")

#### Des filtres pour réduire le bruit

Il existe plusieurs filtres bien connus dont:

##### Filtre médian

Le filtre médian permet de réduire le bruit tout en conservant les contours, il est souvent utilisé pour supprimer le bruit sel et poivre (incursion de pixels noirs et blancs dans l'image) ; chaque pixel est remplacé par la médiane de son voisinage et cela permet de supprimer les valeurs abberantes. Le filtre médian garde le contraste, la luminosité et les contours

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.stack.imgur.com%2Fdw60I.png&f=1&nofb=1&ipt=98a04cc296528ada2699bafee8ef3eafe1af5ab35f7a2d6743a7659fdbe48dc8&ipo=images" title="median"/>

In [ ]:
?scipy.ndimage.filters.median_filter

In [ ]:
import requests 

DownURL = "https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fi.stack.imgur.com%2FJ13Wn.jpg&f=1&nofb=1&ipt=941e7a6b4eee937a7348d2cc6752d0572048342830b97d90f7545151c03dcee3&ipo=images" # choix de l'URL
img_data = requests.get(DownURL).content # télécharger
with open('noise.jpg', 'wb') as handler: # définir le fichier et son chemin
    handler.write(img_data)

In [ ]:
# Paramétrage de la fonction
# La fonction prend principalement en paramètres une image (input), la taille du filtre (size) et le type de padding (mode)

path = "noise.jpg"
image = cv2.imread(path)
gray_mm = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

gray_median = scipy.ndimage.filters.median_filter(gray_mm, size=9)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray_mm,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_median,cmap = "gray")

#### Filtre Gaussien

Le filtre gaussien, comme son nom l'indique suit une distribution gaussienne, c'est-à-dire une loi normale centrée et réduite. Le sigma définit la forme de la cloche, et dans le traitement de l'image cela signifie que le bruit peut être réduit (sigma < 1) ou que le flou peut être accentué (sigma > 1).

Le filtre gaussien vient lisser les imperfection de l'image, les détails et les contours sont atténués.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/10/Generalized_normal_densities.svg/langfr-560px-Generalized_normal_densities.svg.png" title="gauss"/>

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse1.mm.bing.net%2Fth%3Fid%3DOIP.JpX8ONYKxZmIBAHIi1KpjAHaIy%26pid%3DApi&f=1&ipt=f751c30b941d608cdf6715c0829c2abc7c2d6f08aa83e0155fe4bef8ebc48ea9&ipo=images" title="gauss2"/>

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fsupport.cognex.com%2Fdocs%2Fcvl_900%2Fweb%2FEN%2Fcvl_vision_tools%2FContent%2FImages%2F18_8.jpg&f=1&nofb=1&ipt=0ba811ae846f092e40b20e9c7f1d74fc7f1e0f88240a48ec1ce7483ca4f1017b&ipo=images" title="gauss3"/>

In [ ]:
?scipy.ndimage.filters.gaussian_filter

In [ ]:
# Paramétrage de la fonction
# La fonction prend principalement une image (input) et un sigma

gray_g = gray_mm.copy()

gray_gauss = scipy.ndimage.filters.gaussian_filter(gray_g, sigma=3)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray_mm,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_gauss,cmap = "gray")

In [ ]:
# Il existe également une fonction similaire dans la librairie OpenCV

?cv2.GaussianBlur

In [ ]:
# Paramétrage de la fonction
# La fonction prend principalement une image, une taille de filtre (ksize) et un sigma

gray_gcv = gray_mm.copy()

gray_gausscv = cv2.GaussianBlur(gray_gcv, (9,9), 3)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray_mm,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_gausscv,cmap = "gray")

<div class="alert alert-block alert-warning">
<b>Exercice</b>: à partir de l'image et de la détection des visages ci-dessous, il vous faudra flouter ces visages et enregistrer l'image.</div>

In [ ]:
# télécharger une image

import requests 

DownURL = "https://external-content.duckduckgo.com/iu/?u=http%3A%2F%2Fwww.bluebird-electric.net%2Facademia%2Facademia_pictures%2FUniversity_of_Geneva_Universite_de_Geneve_Switzerland_Planet_Solar.jpg&f=1&nofb=1&ipt=47cb55e9bf396b35de8f272546914dbe02979fdd2163fd7e2e15601bec35314c&ipo=images" # choix de l'URL
...

In [ ]:
# import notre image

path = "unige.jpg"
image = cv2.imread(path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)
plt.show()

In [ ]:
# Appliquer un algorithme simple
# https://towardsdatascience.com/viola-jones-algorithm-and-haar-cascade-classifier-ee3bfb19f7d8

grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml") #https://github.com/opencv/opencv/tree/master/data/haarcascades
detected_faces = face_cascade.detectMultiScale(grayscale_image)

In [ ]:
# flouter les visages

In [ ]:
# Dessiner les lignes

for (column, row, width, height) in detected_faces:
    image = cv2.rectangle(image,(column, row),(column + width, row + height),(0, 255, 0),4)
plt.imshow(image)
plt.axis("off")
plt.show()

### Exemple d'application : détecter les contours

Dans une image, un contour se comprend comme la différence d'intensité entre deux pixels. La détection de contours cherche le changement soudain entre deux valeurs de pixel.

Mathématiquement le calcul d'intensité se fait par l'utilisation de dérivées : en somme, l'application de une ou deux dérivées permet d'accentuer le contraste entre deux parties de l'image sensées décrire un contour. 

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ftse2.mm.bing.net%2Fth%3Fid%3DOIP.jeER_gFbhKiY_iGIJrGmjAHaDd%26pid%3DApi&f=1&ipt=1495f4b2280deda1acd3572573157b7525409b810d4a084765dc7d298013d016&ipo=images" title="edge"/>

Il existe plusieurs types de filtres bien connus pour la détection des contours, nous allons en voir certains:

* Prewitt
* Sobel
* Canny
* Laplacien
* Laplacien Gaussien


#### Filtre Canny

Le filtre Canny permet de détecter les contours (sans division horizontale et verticale préalable, comme le Sobel et Prewitt). Il est performant dans la détection (contours faibles et forts) et dans la localisation des contours (faible erreur entre contours détectés et contours réels). Malgré sa performance, son coup d'utilisation est non néglibeable.

En d'autres termes, voilà le processus simplifié du filtre Canny:

<img src="https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Fimage.slidesharecdn.com%2Fe2822eef-6993-4540-9321-65ca5f35eb39-161009120200%2F95%2Fexploring-methods-to-improve-edge-detection-with-canny-algorithm-13-638.jpg%3Fcb%3D1476014597&f=1&nofb=1&ipt=f300f199eaf34361e890d50c9054e9b38c9b25b29d6863942c7ec716bbdbc49b&ipo=images" title="k-means2"/>


In [ ]:
?cv2.Canny

**La fonction prend principalement comme arguments: une image, un seuil bas et un seuil haut**

> * En dessous du seuil bas, le pixel est rejeté
> * En dessus du seuil haut, le pixel est considéré comme un contour
> * Entre, si le pixel est connecté au seuil haut, il est accepté

In [ ]:
# Paramétrage de la fonction

gray_c = gray.copy()

gray_canny = cv2.Canny(gray_c, 100, 200)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_canny,cmap = "gray")

In [ ]:
# Paramétrage de la fonction

gray_c = gray_mm.copy()

gray_canny = cv2.Canny(gray_c, 100, 200)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray_mm,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_canny,cmap = "gray")

In [ ]:
# Ajoutons un filtre median en amont

gray_c = gray_mm.copy()

gray_median = scipy.ndimage.filters.median_filter(gray_mm, size=9)
gray_canny = cv2.Canny(gray_median, 100, 200)

In [ ]:
fig, (ax1,ax2) = plt.subplots(1,2, figsize = (20,10))
                                                 
ax1.set_title ('Image en valeurs de gris')
ax1.imshow(gray_mm,cmap = "gray")

ax2.set_title ('Image transformée')
ax2.imshow(gray_canny,cmap = "gray")